# ① 问题定义

十二生肖分类的本质是图像分类任务，我们采用CNN网络结构进行相关实践。

# ② 数据准备

## 2.1 解压缩数据集

我们将网上获取的数据集以压缩包的方式上传到aistudio数据集中，并加载到我们的项目内。

在使用之前我们进行数据集压缩包的一个解压。

In [3]:
!unzip -q -o data/data76110/signs.zip

## 2.2 数据标注

我们先看一下解压缩后的数据集长成什么样子。

```bash
.
├── test
│   ├── dog
│   ├── dragon
│   ├── goat
│   ├── horse
│   ├── monkey
│   ├── ox
│   ├── pig
│   ├── rabbit
│   ├── ratt
│   ├── rooster
│   ├── snake
│   └── tiger
├── train
│   ├── dog
│   ├── dragon
│   ├── goat
│   ├── horse
│   ├── monkey
│   ├── ox
│   ├── pig
│   ├── rabbit
│   ├── ratt
│   ├── rooster
│   ├── snake
│   └── tiger
└── valid
    ├── dog
    ├── dragon
    ├── goat
    ├── horse
    ├── monkey
    ├── ox
    ├── pig
    ├── rabbit
    ├── ratt
    ├── rooster
    ├── snake
    └── tiger
```

数据集分为train、valid、test三个文件夹，每个文件夹内包含12个分类文件夹，每个分类文件夹内是具体的样本图片。

我们对这些样本进行一个标注处理，最终生成train.txt/valid.txt/test.txt三个数据标注文件。

In [1]:
text = '''__all__ = ['CONFIG', 'get']

CONFIG = {
    'model_save_dir': "./output/zodiac",
    'num_classes': 12,
    'total_images': 7096,
    'epochs': 20,
    'batch_size': 32,
    'image_shape': [3, 224, 224],
    'LEARNING_RATE': {
        'params': {
            'lr': 0.00375             
        }
    },
    'OPTIMIZER': {
        'params': {
            'momentum': 0.9
        },
        'regularizer': {
            'function': 'L2',
            'factor': 0.000001
        }
    },
    'LABEL_MAP': [
        "ratt",
        "ox",
        "tiger",
        "rabbit",
        "dragon",
        "snake",
        "horse",
        "goat",
        "monkey",
        "rooster",
        "dog",
        "pig",
    ]
}

def get(full_path):
    for id, name in enumerate(full_path.split('.')):
        if id == 0:
            config = CONFIG
        
        config = config[name]
    
    return config
'''
fid = open('config.py', 'w')
fid.write(text)
fid.close()

In [4]:
import io
import os
from PIL import Image
from config import get


# 数据集根目录
DATA_ROOT = 'signs'

# 标签List
LABEL_MAP = get('LABEL_MAP')

# 标注生成函数
def generate_annotation(mode):
    # 建立标注文件
    with open('{}/{}.txt'.format(DATA_ROOT, mode), 'w') as f:
        # 对应每个用途的数据文件夹，train/valid/test
        train_dir = '{}/{}'.format(DATA_ROOT, mode)

        # 遍历文件夹，获取里面的分类文件夹
        for path in os.listdir(train_dir):
            # 标签对应的数字索引，实际标注的时候直接使用数字索引
            label_index = LABEL_MAP.index(path)

            # 图像样本所在的路径
            image_path = '{}/{}'.format(train_dir, path)

            # 遍历所有图像
            for image in os.listdir(image_path):
                # 图像完整路径和名称
                image_file = '{}/{}'.format(image_path, image)
                
                try:
                    # 验证图片格式是否ok
                    with open(image_file, 'rb') as f_img:
                        image = Image.open(io.BytesIO(f_img.read()))
                        image.load()
                        
                        if image.mode == 'RGB':
                            f.write('{}\t{}\n'.format(image_file, label_index))
                except:
                    continue


generate_annotation('train')  # 生成训练集标注文件
generate_annotation('valid')  # 生成验证集标注文件
generate_annotation('test')   # 生成测试集标注文件

## 2.3 数据集定义

接下来我们使用标注好的文件进行数据集类的定义，方便后续模型训练使用。

### 2.3.1 导入相关库

In [5]:
import paddle
import numpy as np
from config import get

paddle.__version__

'2.0.1'

### 2.3.2 导入数据集的定义实现

我们数据集的代码实现是在dataset.py中。

In [6]:
text = '''import paddle
import paddle.vision.transforms as T
import numpy as np
from config import get
from PIL import Image

__all__ = ['ZodiacDataset']

# 定义图像的大小
image_shape = get('image_shape')
IMAGE_SIZE = (image_shape[1], image_shape[2])


class ZodiacDataset(paddle.io.Dataset):
    """
    十二生肖数据集类的定义
    """

    def __init__(self, mode='train'):
        """
        初始化函数
        """
        assert mode in ['train', 'test', 'valid'], 'mode is one of train, test, valid.'

        self.data = []

        with open('signs/{}.txt'.format(mode)) as f:
            for line in f.readlines():
                info = line.strip().split('\t')

                if len(info) > 0:
                    self.data.append([info[0].strip(), info[1].strip()])

        if mode == 'train':
            self.transforms = T.Compose([
                T.RandomResizedCrop(IMAGE_SIZE),    # 随机裁剪大小
                T.RandomHorizontalFlip(0.5),        # 随机水平翻转
                T.ToTensor(),                       # 数据的格式转换和标准化 HWC => CHW  
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 图像归一化
            ])
        else:
            self.transforms = T.Compose([
                T.Resize(256),                 # 图像大小修改
                T.RandomCrop(IMAGE_SIZE),      # 随机裁剪
                T.ToTensor(),                  # 数据的格式转换和标准化 HWC => CHW
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])   # 图像归一化
            ])
        
    def __getitem__(self, index):
        """
        根据索引获取单个样本
        """
        image_file, label = self.data[index]
        image = Image.open(image_file)

        if image.mode != 'RGB':
            image = image.convert('RGB')

        image = self.transforms(image)

        return image, np.array(label, dtype='int64')

    def __len__(self):
        """
        获取样本总数
        """
        return len(self.data)
'''
fid = open('dataset.py', 'w')
fid.write(text)
fid.close()

In [7]:
from dataset import ZodiacDataset

### 2.3.3 实例化数据集类

根据所使用的数据集需求实例化数据集类，并查看总样本量。

In [8]:
train_dataset = ZodiacDataset(mode='train')
valid_dataset = ZodiacDataset(mode='valid')

print('训练数据集：{}张；验证数据集：{}张'.format(len(train_dataset), len(valid_dataset)))

训练数据集：7096张；验证数据集：639张


# ③ 模型选择和开发

## 3.1 网络构建

本次我们使用ResNet50网络来完成我们的案例实践。

**1）ResNet系列网络**

![](https://ai-studio-static-online.cdn.bcebos.com/e671828a87424802844246864a66b8100a54e86662b84e269c8758b89625e39b)

**2）ResNet50结构**

![](https://ai-studio-static-online.cdn.bcebos.com/a9045103588d49b09b35855d347f683af9d1926910ad4b639ccf1ec5c36fb7be)

**3）残差区块**

![](https://ai-studio-static-online.cdn.bcebos.com/b2d06daceb7043a8962ba0bd339129731c1c974233ac4baebb074b912f45f80f)

**4）ResNet其他版本**
![](https://ai-studio-static-online.cdn.bcebos.com/d46a2e84bcae40fe95c638b4eb7252b8b9d2767d1c08497da904c3095aa39abb)
![](https://ai-studio-static-online.cdn.bcebos.com/53744138423e4395ae26248aa79d2982fde0321d3ccc44f0b2b764aa54363a98)


In [9]:
# 请补齐模型实例化代码

# network = ?
# resnet50
# network = paddle.vision.models.resnet50(num_classes=get('num_classes'),pretrained=True)
network = paddle.vision.models.vgg16(num_classes=get('num_classes'),pretrained=True)

2021-03-22 15:09:17,426 - INFO - unique_endpoints {''}
2021-03-22 15:09:17,427 - INFO - Downloading vgg16.pdparams from https://paddle-hapi.bj.bcebos.com/models/vgg16.pdparams
100%|██████████| 817517/817517 [00:11<00:00, 68290.64it/s]
2021-03-22 15:09:29,570 - INFO - File /home/aistudio/.cache/paddle/hapi/weights/vgg16.pdparams md5 checking...
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1303: UserWarning: Skip loading for classifier.6.weight. classifier.6.weight receives a shape [4096, 1000], but the expected shape is [4096, 12].
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1303: UserWarning: Skip loading for classifier.6.bias. classifier.6.bias receives a shape [1000], but the expected shape is [12].
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))


**模型可视化**

In [10]:
model = paddle.Model(network)
model.summary((-1, ) + tuple(get('image_shape')))

-------------------------------------------------------------------------------
   Layer (type)         Input Shape          Output Shape         Param #    
     Conv2D-1        [[1, 3, 224, 224]]   [1, 64, 224, 224]        1,792     
      ReLU-1        [[1, 64, 224, 224]]   [1, 64, 224, 224]          0       
     Conv2D-2       [[1, 64, 224, 224]]   [1, 64, 224, 224]       36,928     
      ReLU-2        [[1, 64, 224, 224]]   [1, 64, 224, 224]          0       
    MaxPool2D-1     [[1, 64, 224, 224]]   [1, 64, 112, 112]          0       
     Conv2D-3       [[1, 64, 112, 112]]   [1, 128, 112, 112]      73,856     
      ReLU-3        [[1, 128, 112, 112]]  [1, 128, 112, 112]         0       
     Conv2D-4       [[1, 128, 112, 112]]  [1, 128, 112, 112]      147,584    
      ReLU-4        [[1, 128, 112, 112]]  [1, 128, 112, 112]         0       
    MaxPool2D-2     [[1, 128, 112, 112]]   [1, 128, 56, 56]          0       
     Conv2D-5        [[1, 128, 56, 56]]    [1, 256, 56, 56]   

{'total_params': 134309708, 'trainable_params': 134309708}

# ④ 模型训练和优化

In [11]:
EPOCHS = get('epochs')
BATCH_SIZE = get('batch_size')

# 请补齐模型训练过程代码
def create_optim(parameters):
    step_each_epoch = get('total_images')//get('batch_size')
    lr = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=get('LEARNING_RATE.params.lr'),
                                                    T_max=step_each_epoch*EPOCHS)

    return paddle.optimizer.Momentum(learning_rate=lr,
                                        parameters=parameters,
                                        weight_decay=paddle.regularizer.L2Decay(get('OPTIMIZER.regularizer.factor')))

# 模型训练配置
model.prepare(create_optim(network.parameters()),
                paddle.nn.CrossEntropyLoss(),
                paddle.metric.Accuracy(topk=(1,5)))

# 可视化工具VisualDL的回调函数
visualdl = paddle.callbacks.VisualDL(log_dir='visualdl_log')

# 启动模型训练
model.fit(train_dataset,
          valid_dataset,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          shuffle=True,
          verbose=1,
          save_dir='./chk_points/',
          callbacks=[visualdl])

The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/20


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 222/222 [==============================] - loss: 2.2994 - acc_top1: 0.1064 - acc_top5: 0.4711 - 758ms/step        
save checkpoint at /home/aistudio/chk_points/0
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 20/20 [==============================] - loss: 1.8183 - acc_top1: 0.1471 - acc_top5: 0.5634 - 806ms/step         
Eval samples: 639
Epoch 2/20
step 222/222 [==============================] - loss: 1.5113 - acc_top1: 0.2129 - acc_top5: 0.6554 - 754ms/step        
save checkpoint at /home/aistudio/chk_points/1
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 20/20 [==============================] - loss: 2.2229 - acc_top1: 0.4507 - acc_top5: 0.8623 - 803ms/step         
Eval samples: 639
Epoch 3/20
step 222/222 [==============================] - loss: 1.0639 - acc_top1: 0.4735 - acc_top5: 0.8595 - 760ms/step        
sa

### 模型存储

将我们训练得到的模型进行保存，以便后续评估和测试使用。

In [12]:
model.save(get('model_save_dir'))

# ⑤ 模型评估和测试

## 5.1 批量预测测试

### 5.1.1 测试数据集

In [13]:
predict_dataset = ZodiacDataset(mode='test')
print('测试数据集样本量：{}'.format(len(predict_dataset)))

测试数据集样本量：646


### 5.1.2 执行预测

In [14]:
from paddle.static import InputSpec

# 请补充网络结构

# 模型结构
# network = paddle.vision.models.resnet50(num_classes=get('num_classes'))
network = paddle.vision.models.vgg16(num_classes=get('num_classes'))

# 模型封装
model_2 = paddle.Model(network, inputs=[InputSpec(shape=[-1] + get('image_shape'), dtype='float32', name='image')])

# 请补充模型文件加载代码
# 训练好的模型加载
model_2.load(get('model_save_dir'))

# 模型配置
model_2.prepare()

# 执行预测
result = model_2.predict(predict_dataset)

Predict begin...
step 646/646 [==============================] - 24ms/step         
Predict samples: 646


In [15]:
# 样本映射
LABEL_MAP = get('LABEL_MAP')

# 随机取样本展示
indexs = [2, 38, 56, 92, 100, 303]

for idx in indexs:
    predict_label = np.argmax(result[0][idx])
    real_label = predict_dataset[idx][1]

    print('样本ID：{}, 真实标签：{}, 预测值：{}'.format(idx, LABEL_MAP[real_label], LABEL_MAP[predict_label]))

样本ID：2, 真实标签：pig, 预测值：pig
样本ID：38, 真实标签：pig, 预测值：pig
样本ID：56, 真实标签：dragon, 预测值：dragon
样本ID：92, 真实标签：dragon, 预测值：dragon
样本ID：100, 真实标签：dragon, 预测值：dragon
样本ID：303, 真实标签：ratt, 预测值：ratt


# ⑥ 模型部署

In [16]:
# 可以可视化软件infer/zodiac.pdmodel查看模型结构
model_2.save('infer/zodiac', training=False)